In [ ]:
from automate import Part, PartOptions
from brepmatching.rendering import *
import os
import json
import pandas as pd
from PIL import Image
from tqdm import tqdm
from zipfile import ZipFile

In [ ]:
im_grid = []

with ZipFile('../data/no_tol_v1.zip','r') as zf:
    with zf.open('data/VariationData/all_variations.csv','r') as f:
        variations = pd.read_csv(f)

    for i in tqdm(range(len(variations))):
        var_rec = variations.iloc[i]

        original_path = os.path.join('data/BrepsWithReference',var_rec.ps_orig)
        variation_path = os.path.join('data/BrepsWithReference',var_rec.ps_var)
        match_path = os.path.join('data/Matches',var_rec.matchFile)

        with zf.open(original_path, 'r') as f:
            original_part_data = f.read().decode('utf-8')
            original_part = Part(original_part_data)
        with zf.open(variation_path, 'r') as f:
            variation_part_data = f.read().decode('utf-8')
            variation_part = Part(variation_part_data)

        with zf.open(match_path, 'r') as f:
            match_data = json.load(f)
        
        orig_id_map = {}
        var_id_map = {}
        for id, (_, matches) in enumerate(match_data.items()):
            orig_id = matches['val1']
            var_id = matches['val2']
            orig_id_map[orig_id] = id
            var_id_map[var_id] = id
        orig_ids_unshifted = [orig_id_map.get(f.export_id,len(match_data)) for f in original_part.brep.nodes.faces]
        var_ids_unshifted = [var_id_map.get(f.export_id,len(match_data)) for f in variation_part.brep.nodes.faces]
        shift_map = dict((k,v) for (v,k) in enumerate(set(orig_ids_unshifted + var_ids_unshifted)))
        orig_ids = [shift_map[k] for k in orig_ids_unshifted]
        var_ids = [shift_map[k] for k in var_ids_unshifted]
        orig_mesh_face_ids = [orig_ids[f] for f in original_part.mesh_topology.face_to_topology]
        var_mesh_face_ids = [var_ids[f] for f in variation_part.mesh_topology.face_to_topology]

        im_orig = render_segmented_mesh(original_part.mesh.V, original_part.mesh.F, np.array(orig_mesh_face_ids))
        im_var = render_segmented_mesh(variation_part.mesh.V, variation_part.mesh.F, np.array(var_mesh_face_ids))

        im_grid.append([im_orig, im_var])

Image.fromarray(grid_images(np.array(im_grid)).astype(np.uint8))